In [1]:
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, random_split

In [2]:
# Read the data
DATA_FILENAME = "hbonddata.xlsx"
dataframe_raw = pd.read_excel(DATA_FILENAME,sheet_name='data')
dataframe_raw = dataframe_raw.filter(items=['Acid', 'Base', 'Agroup', 'Aeneg', 'Asize', 'Beneg', 'Bsize', 'Δν', '-ΔH (kJ mol-1)'])
# Drop the rows with NANs. 
dataframe_raw = dataframe_raw.dropna(axis=0, how = 'any')
dataframe_raw.head()

,Acid,Base,Agroup,Aeneg,Asize,Beneg,Bsize,Δν,-ΔH (kJ mol-1)
0,(CCl3)2CHOH,Acetonitrile,COH,3.41,150,2.90,166.0,211.0,22.6
1,(CH2Cl)2CHOH,Acetonitrile,COH,3.41,150,2.90,166.0,102.0,14.2
2,(CHCl2)2CHOH,Acetonitrile,COH,3.41,150,2.90,166.0,174.0,19.8
3,Acetic acid,Pyridine,COOH,3.41,150,2.90,166.0,1000.0,40.0
4,Acetic acid,Dimethyl sulfoxide,COOH,3.41,150,3.41,150.0,840.0,33.5


In [3]:
# Rename the columns
dataframe_raw.rename(columns={'-ΔH (kJ mol-1)': 'ΔH'}, inplace =True)
dataframe_raw.head()

,Acid,Base,Agroup,Aeneg,Asize,Beneg,Bsize,Δν,ΔH
0,(CCl3)2CHOH,Acetonitrile,COH,3.41,150,2.90,166.0,211.0,22.6
1,(CH2Cl)2CHOH,Acetonitrile,COH,3.41,150,2.90,166.0,102.0,14.2
2,(CHCl2)2CHOH,Acetonitrile,COH,3.41,150,2.90,166.0,174.0,19.8
3,Acetic acid,Pyridine,COOH,3.41,150,2.90,166.0,1000.0,40.0
4,Acetic acid,Dimethyl sulfoxide,COOH,3.41,150,3.41,150.0,840.0,33.5


In [4]:
# Check the info of the data
dataframe_raw.describe()

,Aeneg,Asize,Beneg,Bsize,Δν,ΔH
count,318.000000,318.000000,318.000000,318.000000,318.000000,318.000000
mean,3.238428,155.789308,3.172390,157.122642,317.967614,19.464534
std,0.325150,10.909563,0.303832,8.586949,350.151747,11.840464
min,2.480000,150.000000,2.040000,150.000000,1.000000,0.700000
25%,3.410000,150.000000,2.900000,150.000000,109.192487,10.632760
50%,3.410000,150.000000,3.410000,150.000000,218.741953,17.600000
75%,3.410000,150.000000,3.410000,166.000000,364.500000,25.104000
max,3.410000,189.000000,3.410000,177.000000,2300.000000,67.000000


In [26]:
your_name = "Stcike sarpherise Lei h bond project" # at least 5 characters
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.85*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.Δν = dataframe.Δν * ord(rand_str[1])/100.
    # scale target
    dataframe.ΔH = dataframe.ΔH * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['Acid','Base'], axis=1)
    return dataframe

dataframe = customize_dataset(dataframe_raw, your_name)
dataframe

,Agroup,Aeneg,Asize,Beneg,Bsize,Δν,ΔH
266,COH,3.41,150,3.41,150.0,117.160000,12.177000
79,COH,3.41,150,2.90,166.0,280.720000,12.870000
324,HOH,3.41,150,2.90,166.0,480.240000,32.076000
241,NH,2.90,166,2.90,166.0,498.800000,24.453000
156,PhOH,3.41,150,3.41,150.0,138.955647,17.355650
...,...,...,...,...,...,...,...
171,COH,3.41,150,2.90,166.0,421.080000,26.532000
172,PhOH,3.41,150,2.90,166.0,951.200000,37.719000
92,COH,3.41,150,2.90,166.0,350.260170,17.024278
67,PhOH,3.41,150,3.41,150.0,392.080000,33.165000


In [27]:
input_cols = ['Aeneg', 'Asize', 'Beneg', 'Bsize', 'Δν']
categorical_cols = ['Agroup']
output_cols = ['ΔH']

In [28]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[  3.41      , 150.        ,   3.41      , 150.        ,
         117.16      ],
        [  3.41      , 150.        ,   2.9       , 166.        ,
         280.72      ],
        [  3.41      , 150.        ,   2.9       , 166.        ,
         480.24      ],
        ...,
        [  3.41      , 150.        ,   2.9       , 166.        ,
         350.26016967],
        [  3.41      , 150.        ,   3.41      , 150.        ,
         392.08      ],
        [  3.41      , 150.        ,   3.41      , 150.        ,
         444.28      ]]),
 array([[12.177    ],
        [12.87     ],
        [32.076    ],
        [24.453    ],
        [17.3556504],
        [ 4.158    ],
        [13.3791768],
        [40.593168 ],
        [ 6.633    ],
        [14.49756  ],
        [39.35052  ],
        [ 7.92     ],
        [12.7992744],
        [45.149544 ],
        [12.1779504],
        [38.107872 ],
        [43.906896 ],
        [31.0662   ],
        [14.949    ],
        [15.4088352],
        [ 8

In [29]:
inputs = torch.Tensor(inputs_array)
targets = torch.Tensor(targets_array)

dataset = TensorDataset(inputs, targets)
train_ds, val_ds = random_split(dataset, [225, 45])
batch_size = 82

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [30]:
import random
input_size = len(input_cols)
output_size = len(output_cols)
H = 35
criterion = nn.MSELoss(reduction='sum')
class HbondModel(nn.Module):
    def __init__(self):
        super().__init__()
#         self.linear = nn.Linear(input_size, output_size)                  # fill this (hint: use input_size & output_size defined above)
        self.input_linear = nn.Linear(input_size, H)
        self.middle_linear = nn.Linear(H, H)
        self.output_linear = nn.Linear(H, output_size)
               
    def forward(self, xb):     
        h_relu = self.input_linear(xb).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        out = self.output_linear(h_relu)
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = criterion(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = criterion(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [31]:
model = HbondModel()

# Eval algorithm
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

result = evaluate(model, val_loader)
print(result)

{'val_loss': 4026.082763671875}


In [32]:
# Fitting algorithm
def fit(epochs, lr, momentum, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr, momentum)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [33]:
i=0
while i < 50:
    epochs = 90
    lr = 1e-7
    momentum=0.3
    history1 = fit(epochs, lr, momentum, model, train_loader, val_loader)
    i+=1

Epoch [20], val_loss: 3580.0208
Epoch [40], val_loss: 1150.6179
Epoch [60], val_loss: 1104.2694
Epoch [80], val_loss: 886.8676
Epoch [90], val_loss: 812.4584
Epoch [20], val_loss: 832.3939
Epoch [40], val_loss: 798.4988
Epoch [60], val_loss: 789.8549
Epoch [80], val_loss: 718.8917
Epoch [90], val_loss: 779.6071
Epoch [20], val_loss: 752.1075
Epoch [40], val_loss: 740.1486
Epoch [60], val_loss: 705.2037
Epoch [80], val_loss: 711.8788
Epoch [90], val_loss: 721.3656
Epoch [20], val_loss: 736.8622
Epoch [40], val_loss: 710.0642
Epoch [60], val_loss: 691.9840
Epoch [80], val_loss: 758.1166
Epoch [90], val_loss: 693.1184
Epoch [20], val_loss: 726.9534
Epoch [40], val_loss: 711.5714
Epoch [60], val_loss: 712.4976
Epoch [80], val_loss: 667.2184
Epoch [90], val_loss: 729.3417
Epoch [20], val_loss: 702.7180
Epoch [40], val_loss: 660.2286
Epoch [60], val_loss: 729.7037
Epoch [80], val_loss: 701.6681
Epoch [90], val_loss: 705.4083
Epoch [20], val_loss: 679.2469
Epoch [40], val_loss: 723.6345
Epoch

In [34]:
i=0
while i < 50:
    epochs = 70
    momentum=0.1
    lr = 1e-9
    history1 = fit(epochs, lr, momentum, model, train_loader, val_loader)
    i+=1

Epoch [20], val_loss: 694.2618
Epoch [40], val_loss: 662.2474
Epoch [60], val_loss: 691.7355
Epoch [70], val_loss: 662.1429
Epoch [20], val_loss: 661.6611
Epoch [40], val_loss: 688.3156
Epoch [60], val_loss: 710.1476
Epoch [70], val_loss: 709.6465
Epoch [20], val_loss: 661.3975
Epoch [40], val_loss: 686.0246
Epoch [60], val_loss: 661.7451
Epoch [70], val_loss: 708.7631
Epoch [20], val_loss: 662.2126
Epoch [40], val_loss: 662.2391
Epoch [60], val_loss: 662.5270
Epoch [70], val_loss: 683.4066
Epoch [20], val_loss: 683.3770
Epoch [40], val_loss: 683.4886
Epoch [60], val_loss: 709.1913
Epoch [70], val_loss: 662.3774
Epoch [20], val_loss: 709.0323
Epoch [40], val_loss: 683.3334
Epoch [60], val_loss: 661.6613
Epoch [70], val_loss: 683.0842
Epoch [20], val_loss: 683.0771
Epoch [40], val_loss: 708.0115
Epoch [60], val_loss: 707.7217
Epoch [70], val_loss: 707.7292
Epoch [20], val_loss: 661.8415
Epoch [40], val_loss: 663.0531
Epoch [60], val_loss: 661.5287
Epoch [70], val_loss: 663.5209
Epoch [2

In [35]:
# Prediction Algorithm
def validation(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)
    return prediction

# Testing the model with some samples
input, target = val_ds[0]
validation(input, target, model)

Input: tensor([  3.4100, 150.0000,   2.9000, 166.0000, 249.4000])
Target: tensor([18.2160])
Prediction: tensor([16.9904])


tensor([16.9904])

In [36]:
val_value, target = val_ds[:]
predictions = validation(val_value, target, model)

Input: tensor([[   3.4100,  150.0000,    2.9000,  166.0000,  249.4000],
        [   3.4100,  150.0000,    3.4100,  150.0000,  348.0000],
        [   2.9000,  166.0000,    3.4100,  150.0000,  330.6000],
        [   3.4100,  150.0000,    2.9000,  166.0000,  530.1200],
        [   3.4100,  150.0000,    2.9000,  166.0000,  385.2168],
        [   3.4100,  150.0000,    2.9000,  166.0000,  230.6635],
        [   3.4100,  150.0000,    2.9000,  166.0000,  570.7200],
        [   3.4100,  150.0000,    2.9000,  166.0000,  249.9613],
        [   3.4100,  150.0000,    2.9000,  166.0000,  103.2400],
        [   3.4100,  150.0000,    3.4100,  150.0000,  146.1600],
        [   2.4800,  177.0000,    3.4100,  150.0000,   13.9200],
        [   2.9000,  166.0000,    2.9000,  166.0000,   30.1600],
        [   3.4100,  150.0000,    2.9000,  166.0000,  369.4434],
        [   3.4100,  150.0000,    3.4100,  150.0000, 1316.6000],
        [   3.4100,  150.0000,    3.4100,  150.0000,  299.2800],
        [   3.4100

In [43]:
a = torch.linspace(0,47,20)
b = a
fig = plt.figure(figsize=(2.28,2.025))
ax = fig.add_axes([0.18, 0.21, 0.76, 0.76])
ax.scatter(predictions,target, c='tab:blue', s =26, alpha = 0.6, edgecolors='none')
ax.plot(a,b, color='tab:red', alpha = 0.6)
ax.tick_params(axis='x', pad=1.2, labelsize= 8)
ax.tick_params(axis='y', pad=1.2, labelsize= 8)
ax.set_xlabel('Predicted -E (kJ mol$^{-1}$)', fontsize=8)
ax.set_ylabel('Actual -E (kJ mol$^{-1}$)', fontsize=8)
plt.savefig('modeleva3.png',dpi=600)
plt.show()

In [41]:
# Prediction Algorithm
def predict(values, model):
    inputs = values.unsqueeze(0)
    predictions = model(inputs)
    prediction = predictions[0].detach()
    print("The input values are:", inputs)
    print("The H bond energy is:", prediction, 'kJ/mol')
    return prediction

In [42]:
values = torch.tensor([2.9, 166, 2.74, 204, 145])
predict(values, model)

The input values are: tensor([[  2.9000, 166.0000,   2.7400, 204.0000, 145.0000]])
The H bond energy is: tensor([13.2756]) kJ/mol


tensor([13.2756])